In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [6]:
num_iter = 5
xmax = 2.
zmax = 1.
vmax = 1.

for j = 1:num_iter
    # half in contact, half not
    if j%2==0 
        q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, 0.)
    else
        q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, rand(1) .* zmax)
    end
#     q0 = [1.,0.,0.,0.,0.,0.,0.]
#     v0 = vcat(zeros(3),(rand(3) .- .5) .* 2. .* vmax)
    v0 = vcat(zeros(3),(rand(2) .- .5) .* 2. .* vmax,0.)
    u0 = zeros(sim_data.num_v)
    ctrl! = (u,t,x) -> u[:] = 0.

    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    setdirty!(x0)
    
    traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
    qnext = traj[1:sim_data.num_q,end]
    vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,end]
    x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,end]
   
    τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);
    τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

    res = hcat(x_sol_exp,x_sol_ip,x_sol_auglag,x_sol_exp.-x_sol_auglag,x_sol_ip.-x_sol_auglag)

    display(res)
end

Maximum_Iterations_Exceeded


6×5 Array{Float64,2}:
 -2.57234e-9  -1.84397e-10   3.48066e-7   -3.50638e-7   -3.4825e-7  
 -1.04525e-9  -2.01195e-10  -0.000310114   0.000310113   0.000310114
  6.84516e-9  -2.28911e-10   0.00117633   -0.00117632   -0.00117633 
  7.4515e-11  -2.08652e-10  -0.000512272   0.000512272   0.000512272
  0.521054     8.79522       0.479322      0.0417316     8.3159     
  5.51841e-9  -2.05198e-9   -4.66044e-8    5.21228e-8    4.45525e-8 

Solve_Succeeded


6×5 Array{Float64,2}:
  0.500013    -3.00445e-17   0.0835197    0.416494    -0.0835197 
 -4.46891e-9   6.49401e-23   6.20098e-7  -6.24567e-7  -6.20098e-7
 -4.11446e-9   3.89486e-23   6.81916e-7  -6.8603e-7   -6.81916e-7
  2.22989e-8   7.42702e-23  -0.00668819   0.00668822   0.00668819
  0.838663     9.77423       0.838663     6.70263e-9   8.93556   
  1.00003      0.0           0.153567     0.84646     -0.153567  

Solve_Succeeded
Solve_Succeeded


...

Solved_To_Acceptable_Level
Solve_Succeeded


6×5 Array{Float64,2}:
 -2.02659e-9  9.88982e-24  -3.32376e-7  3.30349e-7   3.32376e-7
 -1.24371e-9  0.0          -3.48364e-7  3.47121e-7   3.48364e-7
  0.500014    0.0           0.0397235   0.460291    -0.0397235 
  2.3013e-8   0.0          -1.27449e-5  1.27679e-5   1.27449e-5
  0.846815    9.76506       0.846815    1.33064e-8   8.91825   
  1.00003     0.0           0.0794206   0.920608    -0.0794206 

Solve_Succeeded
Solve_Succeeded


6×5 Array{Float64,2}:
 -6.62028e-10   1.36784e-15  -0.000281416   0.000281416   0.000281416
 -5.47719e-10  -6.95257e-22   0.00140296   -0.00140296   -0.00140296 
 -5.42859e-10  -5.98186e-22  -5.23104e-8    5.17676e-8    5.23104e-8 
 -6.58044e-10  -6.45266e-22  -0.00027868    0.00027868    0.00027868 
  8.3448       10.6443        0.413154      7.93165      10.2312     
 -6.03233e-9    0.0           0.001166     -0.001166     -0.001166   

Solve_Succeeded
Solve_Succeeded
